In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import warnings
import dfview # https://github.com/xslyr/DataOverview

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


df = pd.read_csv('General_Files/train.csv')
df = df.drop(['Id'],axis=1)
print('\n\nDataframe de Treinamento: train.csv ({} linhas e {} colunas)'.format(df.shape[0],df.shape[1]))

dfinfo = dfview.DataOverview(df)



Dataframe de Treinamento: train.csv (1460 linhas e 80 colunas)
Distribuição dos tipos das colunas do dataframe são:

  34 colunas do tipo int64
  3 colunas do tipo float64
  43 colunas do tipo object
  0 colunas do tipo bool
  0 colunas do tipo datetime64


### Describe das Colunas Tipo Object

In [2]:
obj_columns = dfinfo.show('object')
obj_columns.iloc[1:,:]

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1460,91,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1452,1460,1460,1460,1423,1423,1422,1423,1422,1460,1460,1460,1459,1460,1460,770,1379,1379,1379,1379,1460,7,281,54,1460,1460
null,0,0,1369,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,37,37,38,37,38,0,0,0,1,0,0,690,81,81,81,81,0,1453,1179,1406,0,0
mode,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,PConc,TA,TA,No,Unf,Unf,GasA,Ex,Y,SBrkr,TA,Typ,NaN,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal
n_mode,1151,1454,50,925,1311,1459,1052,1382,225,1260,1445,1220,726,1141,1434,515,504,864,906,1282,647,649,1311,953,430,1256,1428,741,1365,1334,735,1360,380,870,605,1311,1326,1340,3,157,49,1267,1198


Observamos através do dataset acima apresentado que:
  - Descartaremos as colunas abaixo devido ao alto índice de valores nulos. Seguem seus nomes e as porcentagens
    - Alley 93% 
    - PoolQC 99%
    - Fence 80%
    - MiscFeature 96%
    
    
  - A linha n_moda denotam que certas colunas estão com uma alta % de dados em uma única categoria. Portanto separaremos as colunas em 2 blocos e confirmaremos depois a eliminação delas após conferir a distribuição de cada 1:

    - Bloco 1 Dados com Alta % de Enviesamento (acima de 80%) 
    - Bloco 2 Dados com Média % de Enviesamento (entre 50% e 80%)
    - Bloco 3 Dados com Baixa % de Enviesamento (abaixo de 50%)


In [3]:
bias_ratio=(.8,.5)

objs = dfinfo.show('object')
dropped_columns = ['Alley','PoolQC','Fence','MiscFeature']
high_bias = {col: round(objs[col]['n_mode']/objs[col]['count'],2) for col in objs if (objs[col]['n_mode']/objs[col]['count'] > bias_ratio[0]) and (col not in dropped_columns)}
medium_bias = {col:round(objs[col]['n_mode']/objs[col]['count'],2) for col in objs if (objs[col]['n_mode']/objs[col]['count'] > bias_ratio[1]) and (col not in dropped_columns+list(high_bias.keys()))}
low_bias = {col:round(objs[col]['n_mode']/objs[col]['count'],2) for col in objs if (objs[col]['n_mode']/objs[col]['count'] <= bias_ratio[1] ) and (col not in dropped_columns)}

#### Analise da distribuição do Bloco 1 - ( high_bias )

In [4]:
print('\nAbaixo segue as colunas e a porcentagem a % de ocorrencias da Moda em todo conjunto de dados.\n')
for i in high_bias:
    print('\t{}\t{}'.format(high_bias[i],i))
    
print('\n\nA seguir temos um detalhamento de como esta a distribuição de cada 1 delas.')
dfinfo.obj_distrib(column_list=high_bias.keys())


Abaixo segue as colunas e a porcentagem a % de ocorrencias da Moda em todo conjunto de dados.

	1.0	Street
	0.9	LandContour
	1.0	Utilities
	0.95	LandSlope
	0.86	Condition1
	0.99	Condition2
	0.84	BldgType
	0.98	RoofMatl
	0.88	ExterCond
	0.92	BsmtCond
	0.88	BsmtFinType2
	0.98	Heating
	0.93	CentralAir
	0.91	Electrical
	0.93	Functional
	0.95	GarageQual
	0.96	GarageCond
	0.92	PavedDrive
	0.87	SaleType
	0.82	SaleCondition


A seguir temos um detalhamento de como esta a distribuição de cada 1 delas.


,Street,LandContour,Utilities,LandSlope,Condition1,Condition2,BldgType,RoofMatl,ExterCond,BsmtCond,BsmtFinType2,Heating,CentralAir,Electrical,Functional,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,null 0,null 0,null 0,null 0,null 0,null 0,null 0,null 0,null 0,null 37,null 38,null 0,null 0,null 1,null 0,null 81,null 81,null 0,null 0,null 0
1,Pave 1454,Lvl 1311,AllPub 1459,Gtl 1382,Norm 1260,Norm 1445,1Fam 1220,CompShg 1434,TA 1282,TA 1311,Unf 1256,GasA 1428,Y 1365,SBrkr 1334,Typ 1360,TA 1311,TA 1326,Y 1340,WD 1267,Normal 1198
2,Grvl 6,Bnk 63,NoSeWa 1,Mod 65,Feedr 81,Feedr 6,TwnhsE 114,Tar&Grv 11,Gd 146,Gd 65,Rec 54,GasW 18,N 95,FuseA 94,Min2 34,Fa 48,Fa 35,N 90,New 122,Partial 125
3,,HLS 50,,Sev 13,Artery 48,Artery 2,Duplex 52,WdShngl 6,Fa 28,Fa 45,LwQ 46,Grav 7,,FuseF 27,Min1 31,Gd 14,Gd 9,P 30,COD 43,Abnorml 101
4,,Low 36,,,RRAn 26,RRNn 2,Twnhs 43,WdShake 5,Ex 3,Po 2,BLQ 33,Wall 4,,FuseP 3,Mod 15,Ex 3,Po 7,,ConLD 9,Family 20
5,,,,,PosN 19,PosN 2,2fmCon 31,Metal 1,Po 1,,ALQ 19,OthW 2,,Mix 1,Maj1 14,Po 3,Ex 2,,ConLI 5,Alloca 12
6,,,,,RRAe 11,PosA 1,,Membran 1,,,GLQ 14,Floor 1,,,Maj2 5,,,,ConLw 5,AdjLand 4
7,,,,,PosA 8,RRAn 1,,Roll 1,,,,,,,Sev 1,,,,CWD 4,
8,,,,,RRNn 5,RRAe 1,,ClyTile 1,,,,,,,,,,,Oth 3,
9,,,,,RRNe 2,,,,,,,,,,,,,,Con 2,


Ao conferir a distribuição dos dados acima, em nosso modelo inicial, adotaremos as colunas que contém no máximo 90% de enviesamento e descartaremos as restantes.


In [5]:
selected_obj_columns = [i for i in high_bias if high_bias[i]<=.9]
print('As colunas que serão mantidas são:\n\n{}\n'.format(selected_obj_columns))

As colunas que serão mantidas são:

['LandContour', 'Condition1', 'BldgType', 'ExterCond', 'BsmtFinType2', 'SaleType', 'SaleCondition']



#### Analise da distribuição do Bloco 2 - ( medium_bias )

In [6]:
print('\nAbaixo segue as colunas e a porcentagem a % de ocorrencias da Moda em todo conjunto de dados.\n')
for i in medium_bias:
    print('\t{}\t{}'.format(medium_bias[i],i))
    
print('\n\nA seguir temos um detalhamento de como esta a distribuição de cada 1 delas.')
dfinfo.obj_distrib(column_list=medium_bias.keys())


Abaixo segue as colunas e a porcentagem a % de ocorrencias da Moda em todo conjunto de dados.

	0.79	MSZoning
	0.63	LotShape
	0.72	LotConfig
	0.78	RoofStyle
	0.6	MasVnrType
	0.62	ExterQual
	0.67	BsmtExposure
	0.51	HeatingQC
	0.5	KitchenQual
	0.63	GarageType


A seguir temos um detalhamento de como esta a distribuição de cada 1 delas.


,MSZoning,LotShape,LotConfig,RoofStyle,MasVnrType,ExterQual,BsmtExposure,HeatingQC,KitchenQual,GarageType
0,null 0,null 0,null 0,null 0,null 8,null 0,null 38,null 0,null 0,null 81
1,RL 1151,Reg 925,Inside 1052,Gable 1141,None 864,TA 906,No 953,Ex 741,TA 735,Attchd 870
2,RM 218,IR1 484,Corner 263,Hip 286,BrkFace 445,Gd 488,Av 221,TA 428,Gd 586,Detchd 387
3,FV 65,IR2 41,CulDSac 94,Flat 13,Stone 128,Ex 52,Gd 134,Gd 241,Ex 100,BuiltIn 88
4,RH 16,IR3 10,FR2 47,Gambrel 11,BrkCmn 15,Fa 14,Mn 114,Fa 49,Fa 39,Basment 19
5,C (all) 10,,FR3 4,Mansard 7,,,,Po 1,,CarPort 9
6,,,,Shed 2,,,,,,2Types 6


#### Analise da distribuição do Bloco 3 - ( low_bias )

In [7]:
print('\nAbaixo segue as colunas e a porcentagem a % de ocorrencias da Moda em todo conjunto de dados.\n')
for i in low_bias:
    print('\t{}\t{}'.format(low_bias[i],i))
    
print('\n\nA seguir temos um detalhamento de como esta a distribuição de cada 1 delas.')
dfinfo.obj_distrib(column_list=low_bias.keys())


Abaixo segue as colunas e a porcentagem a % de ocorrencias da Moda em todo conjunto de dados.

	0.15	Neighborhood
	0.5	HouseStyle
	0.35	Exterior1st
	0.35	Exterior2nd
	0.44	Foundation
	0.46	BsmtQual
	0.3	BsmtFinType1
	0.49	FireplaceQu
	0.44	GarageFinish


A seguir temos um detalhamento de como esta a distribuição de cada 1 delas.


,Neighborhood,HouseStyle,Exterior1st,Exterior2nd,Foundation,BsmtQual,BsmtFinType1,FireplaceQu,GarageFinish
0,null 0,null 0,null 0,null 0,null 0,null 37,null 37,null 690,null 81
1,NAmes 225,1Story 726,VinylSd 515,VinylSd 504,PConc 647,TA 649,Unf 430,Gd 380,Unf 605
2,CollgCr 150,2Story 445,HdBoard 222,MetalSd 214,CBlock 634,Gd 618,GLQ 418,TA 313,RFn 422
3,OldTown 113,1.5Fin 154,MetalSd 220,HdBoard 207,BrkTil 146,Ex 121,ALQ 220,Fa 33,Fin 352
4,Edwards 100,SLvl 65,Wd Sdng 206,Wd Sdng 197,Slab 24,Fa 35,BLQ 148,Ex 24,
5,Somerst 86,SFoyer 37,Plywood 108,Plywood 142,Stone 6,,Rec 133,Po 20,
6,Gilbert 79,1.5Unf 14,CemntBd 61,CmentBd 60,Wood 3,,LwQ 74,,
7,NridgHt 77,2.5Unf 11,BrkFace 50,Wd Shng 38,,,,,
8,Sawyer 74,2.5Fin 8,WdShing 26,Stucco 26,,,,,
9,NWAmes 73,,Stucco 25,BrkFace 25,,,,,


Todas demais colunas do bloco 2(medium_bias) e 3(high_bias) será inclusa em nosso modelo inicial. 

In [8]:
selected_obj_columns = list(medium_bias) + list(low_bias)
print('De um total de {} colunas do tipo object, utilizaremos apenas {} delas.\n'
      .format(objs.shape[1], len(selected_obj_columns)))

De um total de 43 colunas do tipo object, utilizaremos apenas 19 delas.



In [9]:
cols2_categorize = ['MSSubClass']

selected_obj_columns += list(cols2_categorize)

print('Considerando que na análise das colunas int precisamos transformar 3 colunas em variáveis categóricas, a nossa listagem de {} colunas do tipo object será a seguinte:\n'.format(len(selected_obj_columns)))
for i in selected_obj_columns:
    print('\t{}'.format(i))


Considerando que na análise das colunas int precisamos transformar 3 colunas em variáveis categóricas, a nossa listagem de 20 colunas do tipo object será a seguinte:

	MSZoning
	LotShape
	LotConfig
	RoofStyle
	MasVnrType
	ExterQual
	BsmtExposure
	HeatingQC
	KitchenQual
	GarageType
	Neighborhood
	HouseStyle
	Exterior1st
	Exterior2nd
	Foundation
	BsmtQual
	BsmtFinType1
	FireplaceQu
	GarageFinish
	MSSubClass


#### Análise das ocorrências da Distribuição dos dados, para correta aplicação do Encoding

A idéia nesta etapa é conferir se cada coluna do nosso dataframe contém todas as ocorrências daquela categoria, para possivelmente tratá-las.

Portanto iremos nessa seção, listar as ocorrências de cada "coluna Object" e compará-las com o dicionário de dados para conferir se há pelo menos 1 ocorrencia de cada categoria em todas as colunas.


In [10]:
for col in selected_obj_columns:
    categs = df[col].sort_values().unique()
    print('\n  Column: {}\n  Qtd de Categorias Ocorridas: {}\n  Categories List: {}\n\n'.format(col,len(categs),categs))
        


  Column: MSZoning
  Qtd de Categorias Ocorridas: 5
  Categories List: ['C (all)' 'FV' 'RH' 'RL' 'RM']



  Column: LotShape
  Qtd de Categorias Ocorridas: 4
  Categories List: ['IR1' 'IR2' 'IR3' 'Reg']



  Column: LotConfig
  Qtd de Categorias Ocorridas: 5
  Categories List: ['Corner' 'CulDSac' 'FR2' 'FR3' 'Inside']



  Column: RoofStyle
  Qtd de Categorias Ocorridas: 6
  Categories List: ['Flat' 'Gable' 'Gambrel' 'Hip' 'Mansard' 'Shed']



  Column: MasVnrType
  Qtd de Categorias Ocorridas: 5
  Categories List: ['BrkCmn' 'BrkFace' 'None' 'Stone' nan]



  Column: ExterQual
  Qtd de Categorias Ocorridas: 4
  Categories List: ['Ex' 'Fa' 'Gd' 'TA']



  Column: BsmtExposure
  Qtd de Categorias Ocorridas: 5
  Categories List: ['Av' 'Gd' 'Mn' 'No' nan]



  Column: HeatingQC
  Qtd de Categorias Ocorridas: 5
  Categories List: ['Ex' 'Fa' 'Gd' 'Po' 'TA']



  Column: KitchenQual
  Qtd de Categorias Ocorridas: 4
  Categories List: ['Ex' 'Fa' 'Gd' 'TA']



  Column: GarageType
  Qtd de Cat

Confirmando as possibilidades com o dicionário de dados, seguem as colunas e categorias faltantes:

    MSZoning - ['A','I','RP']
    MasVnrType - ['CBlock']
    ExterQual - ['Po']
    KitchenQual - ['Po']
    Exterior1st - ['Other','PreCast']
    Exterior2nd - ['PreCast']
    
    
Ao analisar o dataset, confirmamos que as colunas abaixo estão com os valores nan erroneamente classificados e portanto iremos substituílos pela string 'NA'. Seguem tais colunas.

    BsmtExposure - ['NA']
    GarageType - ['NA']
    BsmtQual - ['Po','NA']
    BsmtFinType1 - ['NA']
    FireplaceQu - ['NA']
    GarageFinish - ['NA']
    
    
A coluna abaixo além de ser transformada para variável categorica, precisaremos tratar o dado faltante:

    MSSubClass - ['150']

